In [ ]:
# japanize-matplotlibはkaggleのデフォライブラリーではないので別途インストールする必要があります
## kaggle kernelで使用するようにするための方法：https://bit.ly/3kViqBX

!pip install japanize-matplotlib

In [ ]:
# 必要なライブラリのimport
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
import os
import re
import glob
import shutil
from pathlib import Path

# matplotlibの日本語化対応
import japanize_matplotlib

# データフレーム表示用関数
from IPython.display import display

# 表示オプション調整
# numpyの浮動小数点の表示精度
np.set_printoptions(suppress=True, precision=4)

# pandasでの浮動小数点の表示精度
pd.options.display.float_format = '{:.4f}'.format

# データフレームですべての項目を表示
pd.set_option("display.max_columns",None)

# グラフのデフォルトフォント指定
plt.rcParams["font.size"] = 14

# 乱数の種
random_seed = 123

In [ ]:
# セッション時間表示

# https://github.com/nyk510/vivid/blob/master/vivid/utils.py
from contextlib import contextmanager
from time import time

class Timer:
    def __init__(self, logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None, sep=' '):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)

# 今回のコンペについて
## コンペの外観

* [Home Credit とは](https://www.homecredit.net/)：信用力が足りずに融資を受けることができない顧客にも融資を行う会社のことです。
* 目的：個人のクレジット情報や以前の応募情報などから、各データが債務不履行になるかどうかを予測する問題です。


* 評価方法：[AUC (Aread Under the ROC curve)](https://blog.kikagaku.co.jp/roc-auc)
  * 閾値を変化させた際に描かれるROC曲線の下の面積(=AUC)を0~1の範囲で判断し、1に近づくほど良いとされます。
  * つまり positive or negative, Yes or No みたいなのをきちんと分類されていれば良いとされます。



# 機械学習とは
機械学習とは、ざっと言ってしまうとあるデータ X を入力として対応する予測値 y を取り出すような対応関係を作成することです。

例：タイタニック号で、乗客が生きるか死ぬかを予測する問題だと X は乗客の年齢, 性別, 船室のグレード… など乗客に紐づく情報のことを指します。通常、この情報のことを特徴量とよびます。

特徴量 X と 予測値 y が用意できれば学習用データ (X - y の関係がわかっているデータ) を元にして X をいれて y になるようにモデルを調整する。この調整の段階を学習とよびます。学習には様々なアルゴリズムがあるが、X, y を用意しなくてはならない部分は基本的に変わらないです。

# データ読み込み

In [ ]:
# ローカルで使うときはこっち！

# # input_dir（input directory） を作ります
# current_note_path = os.path.dirname(os.path.abspath('__file__'))
# INPUT_DIR = os.path.join(current_note_path, "data")

# # INPUT_DIRがまだ作られていなければ作成
# if not os.path.isdir(INPUT_DIR):
#     os.mkdir(INPUT_DIR)

# # csvファイルを `data` ディレクトリ（=フォルダー） に移動させます
# unique_dir_names = []
# # ローカルで使用する場合は f'{current_note_path}' に変えます
# for f in Path(f'{current_note_path}').rglob('*.csv'):
#     unique_dir_names.append(f)

# for file in list(set(unique_dir_names)):
#     print(f'moved file: {file}')
#     shutil.move(f'{file}', f'{INPUT_DIR}')

In [ ]:
# kaggle notebookで使うときはこっち！

# INPUT_DIR を指定します
INPUT_DIR = '../input/home-credit-default-risk'

# output_dir(output directory) を作ります
## notbookがあるディレクトリパスを `current_note_path` に渡します
current_note_path = os.path.dirname(os.path.abspath('__file__'))
OUTPUT_DIR = os.path.join(current_note_path, 'outputs')

# OUTPUT_DIRがまだ作られていなければ作成
if not os.path.isdir(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

In [ ]:
# csv を読み取る関数を設定したあげると、pathや拡張子を書かずに読み込めるので入力が楽になります（Kaggle notebookだとされないかも）
def read_csv(name, **kwrgs):
    path = os.path.join(INPUT_DIR, name + '.csv')
    print(f'Load: {path}')
    return pd.read_csv(path, **kwrgs)

<b>application_{train|test}</b>

* 基本となるファイルです。顧客ごとに1つずつローンの情報とtrainにはデフォルトしたかという情報(Target)が含まれています。
* ER図で見ても分かる通り、`SK_ID_CURR` で `previous_appliation.csv` や `bureau.csv` と紐づけられます。
* application_testデータはtestデータとして使用されます。

>以下のテーブルデータをtrain_df, test_dfに正確に加えるためにはSK_ID_CURRごとにデータをまとめる必要があります

<b>bureau_balance</b>

* bureau.csvの毎月の残高データです。bureauと組み合わせて使います。

<b>bureau</b>

* 調査局のデータで、全ての顧客が過去に借りた他の金融機関からのローン情報となります。他の金融機関から Home Credit 社に情報提供されたものです。

<b>credit_card_balance</b>

* Home Credit 社のもつ申請者の月次クレジットカード残高のスナップショットになります。

<b>installments_payments</b>

* サンプルローンに関連する Home Credit 社にある過去実績になります。

<b>POS_CASH_balance</b>

* Home Credit 社のもつ申請者の月次クレジットカード残高を販売拠点ごとにまとめたものです。

<b>previous_application</b>

* 申請者の Home Credit 社でのローン履歴となります。

<b>sample_submission</b>

* サンプルcsvです。



In [ ]:
# application_train.csv
app_train = read_csv('application_train')
app_test = read_csv('application_test')
bureau_balance = read_csv('bureau_balance')
bureau = read_csv('bureau')
credit_balance = read_csv('credit_card_balance')
# カラムの説明csvは手元でutf-8形式に直してあげないと読み込めなさそうです。
# カラムの日本語説明の記事：https://www.ritzcolor.net/?p=235
# column_desc = read_csv('HomeCredit_columns_description')
instal_payment = read_csv('installments_payments')
pos_cash = read_csv('POS_CASH_balance')
prev_app = read_csv('previous_application')
samp_sub = read_csv('sample_submission')

In [ ]:
# 以下のカラムは頻出で、毎回入力するのはめんどくさいので、ポップアップされるように定義します
SK_ID_CURR = 'SK_ID_CURR'
SK_ID_PREV = 'SK_ID_PREV'
SK_ID_BUREAU = 'SK_ID_BUREAU'

各データのサイズを見てみます

## データ確認

app_train, app_testを例に見てみると、
* 各ユーザーで1行
* app_testには `TARGET` が抜けているとわかる
* 欠損値が散見される
* カテゴリ関数がある
* 日付が時間差表記されている
* カラム多すぎるし、何なのかわからん (* ただありがたいことに `HomeCredit_columns_description.csv` で説明がされている)

In [ ]:
app_train.head()

In [ ]:
app_test.head()

In [ ]:
# データの統計量
display(app_train.describe())

bureauも見てみる


In [ ]:
bureau.head()

In [ ]:
bureau_balance.head()

In [ ]:
credit_balance.head()

In [ ]:
prev_app.head()

In [ ]:
instal_payment.head()

In [ ]:
pos_cash.head()

欠損値状況を見てみると、かなり欠損値が多いカラムがあることが伺えます

In [ ]:
# 欠損値の確認関数
def missing_values_summary(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(columns = {0 : 'mis_val_count', 1 : 'mis_val_percent'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,1] != 0].sort_values('mis_val_percent', ascending=False).round(1)
    print ("カラム数：" + str(df.shape[1]) + "\n" + "欠損値のカラム数： " + str(mis_val_table_ren_columns.shape[0]))
    return mis_val_table_ren_columns

In [ ]:
missing_values_summary(app_train)

In [ ]:
missing_values_summary(bureau)

In [ ]:
missing_values_summary(prev_app)

## スキーマ（構造）の理解

各データのサイズを見てみます

In [ ]:
# サイズ
print(f'Size of application_train: {app_train.shape}')
print('>Unique counts of current loan ID:', app_train[SK_ID_CURR].nunique())
print(f'Size of application_test: {app_test.shape}')
print('>Unique counts of current loan ID:', app_test[SK_ID_CURR].nunique())
print('--------')
print(f'Size of bureau_balance: {bureau_balance.shape}')
print('>Unique counts of bureau ID:', bureau_balance[SK_ID_BUREAU].nunique())
print(f'Size of bureau: {bureau.shape}')
print('>Unique counts of bureau ID:', bureau[SK_ID_BUREAU].nunique())
print('--------')
print(f'Size of previous_application: {prev_app.shape}')
print('>Unique counts of current loan ID:', prev_app[SK_ID_CURR].nunique())
print('>Unique counts of previous loan ID:', prev_app[SK_ID_PREV].nunique())
print('--------')
print(f'Size of credit_card_balance: {credit_balance.shape}')
print('>Unique counts of current loan ID:', prev_app[SK_ID_CURR].nunique())
print('>Unique counts of previous loan ID:', prev_app[SK_ID_PREV].nunique())
print('--------')
print(f'Size of installments_payments: {instal_payment.shape}')
print('>Unique counts of current loan ID:', instal_payment[SK_ID_CURR].nunique())
print('>Unique counts of previous loan ID:', instal_payment[SK_ID_PREV].nunique())
print('--------')
print(f'Size of POS_CASH_balance: {pos_cash.shape}')
print('>Unique counts of current loan ID:', pos_cash[SK_ID_CURR].nunique())
print('>Unique counts of previous loan ID:', pos_cash[SK_ID_PREV].nunique())
print('--------')
# サブミットするサンプルcsv
print(f'Size of sample_submission: {samp_sub.shape}')


データフレームの特徴を自動的に作ってくれる pandas-profiling を利用する方法もある\
しかしカラム数が多すぎると、ファイルサイズが大きくなり、表示失敗しやすくなるため注意(？)

In [ ]:
# https://qiita.com/wakame1367/items/39faf5d91e20a5cf5772#_reference-a12e589891bdd7bf36e5
# from pandas_profiling import ProfileReport

# カラム数が多いせいか、htmlファイルのロードで失敗することがあるので、dataframe を半分にします
# app_train_first_half = app_train.iloc[:, :60]
# app_train_second_half = app_train.iloc[:, 60:]
# app_train_second_half = pd.concat([app_train.iloc[:, :2], app_train_second_half], axis=1)

# 分割したカラムそれぞれでレポートを作成します
# report1 = ProfileReport(app_train_first_half)
# report1.to_file(os.path.join(OUTPUT_DIR, "report_of_application_train_first_half.html"))
# report2 = ProfileReport(app_train_second_half)
# report2.to_file(os.path.join(OUTPUT_DIR, "report_of_application_train_second_half.html"))

## 学習データ(=入力データ & 正解データ)の作成

ある程度データを理解したところで学習データを作っていきましょう

* 基本方針として、`app_train` でとりあえず使えそうなカラムと、その他のテーブルデータの基本統計量をマージして使用します
* Light GBMといったGBDT系のモデルを使用します
* 欠損値は一旦そのままにします
* カテゴリデータはラベルエンコーディングします

In [ ]:
credit_balance.groupby([SK_ID_CURR], as_index=False).mean()

In [ ]:
# 各ｄｆの基本統計量のうち、一旦平均値を取って集計する -> SK_ID_CURRでユニークな値が生まれた
cd_b = credit_balance.groupby([SK_ID_CURR], as_index=False).mean()
ins_p = instal_payment.groupby([SK_ID_CURR], as_index=False).mean()
pos_c = pos_cash.groupby([SK_ID_CURR], as_index=False).mean()
prv_a = prev_app.groupby([SK_ID_CURR], as_index=False).mean()

# SK_ID_PREV は不要なため削除する
cd_b = cd_b.drop(SK_ID_PREV, axis=1)
ins_p = ins_p.drop(SK_ID_PREV, axis=1)
pos_c = pos_c.drop(SK_ID_PREV, axis=1)
prv_a = prv_a.drop(SK_ID_PREV, axis=1)

# カラム名が重複しないように名称を変更する
cd_b2 = cd_b.rename(columns={'MONTHS_BALANCE':'MONTHS_BALANCE_CREDIT', 'SK_DPD' : 'SK_DPD_CRE_CREDIT', 'SK_DPD_DEF' : 'SK_DPD_DEF_CREDIT'})
prv_a2 = prv_a.rename(columns={'AMT_CREDIT':'AMT_CREDIT_PRE_APP', 'AMT_ANNUITY' : 'AMT_ANNUITY_PRE_APP', 'AMT_GOODS_PRICE' : 'AMT_GOODS_PRICE_PRE_APP'})


前処理（カテゴリ変数の変換）

カテゴリデータは基本的にそのまま特徴量として扱えないので、数値化する必要があります。\
カテゴリ変数の変換にはいくつもの手法がありますが、よくやるものでもデータや使うモデルによって向き不向きがあるので気をつけましょう。

* One-Hot Encoding -> gbdt系以外（線形モデル etc..）におすすめ
* Label Encoding-> gbdt系 にもおすすめ\
[【sklearn】LabelEncoderの使い方を丁寧に](https://gotutiyan.hatenablog.com/entry/2020/09/08/122621)
* Target Encoding -> gbdt系にはより効果的らしい\
[Target Encoding はなぜ有効なのか](https://speakerdeck.com/hakubishin3/target-encoding-hanazeyou-xiao-nafalseka)

In [ ]:
# LabelEncoder()は，文字列や数値で表されたラベルを，0~(ラベル種類数-1)までの数値に変換してくれるものです
from sklearn.preprocessing import LabelEncoder

tmp_app_train = app_train.copy()
tmp_app_test = app_test.copy()
# 今回は簡単に使えそうなカラムをラベルエンコーディングします
for c in ['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE']:
    # LabelEncoderを宣言します
    le = LabelEncoder()
    # ラベルとラベルIDの対応づけを行います。positiveは0にしよう，みたいなことを決めます
    le.fit(tmp_app_train[c].fillna('NA'))

    # 学習データとテストデータそれぞれのdf内のカラムを変換します
    tmp_app_train[c] = le.transform(tmp_app_train[c].fillna('NA'))
    tmp_app_test[c] = le.transform(tmp_app_test[c].fillna('NA'))

In [ ]:
tmp_app_train = tmp_app_train.drop(['FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE'], axis=1)
tmp_app_test = tmp_app_test.drop(['FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE'], axis=1)

In [ ]:
app_train.head()

In [ ]:
# app_trainと比べて、`Target` カラムがなくなっていることがわかると思います
tmp_app_train.head()

テーブルデータをマージします

In [ ]:
#　マージ
## `instalments_payments` に `credit_card_balance` テーブルをマージします
tmp1 = pd.merge(ins_p, cd_b2, on=SK_ID_CURR, how='left')
## `tmp1` に `pos_c` テーブルをマージします
tmp2 = pd.merge(tmp1, pos_c, on=SK_ID_CURR, how='left')
## `tmp2` に `previous_application` テーブルをマージします
tmp3 = pd.merge(tmp2, prv_a2, on=SK_ID_CURR, how='left')

## train と test にもマージします
### 今回は app_{train|test} のカラムの削除を敢えてせずに全て使用します。
train = pd.merge(tmp_app_train, tmp3, on=SK_ID_CURR, how='left')
test = pd.merge(tmp_app_test, tmp3, on=SK_ID_CURR, how='left')

予測対象となる `TARGET`カラムを抜き出し、 `y` とします

同時にtrainから `TARGET`カラムを削除します

In [ ]:
y_train = train['TARGET']

X_train = train.drop('TARGET', axis=1)

X_test = test.copy()


In [ ]:
# numpy 配列に直します
X, y = X_train.values, y_train.values
X_test2 = X_test.values

In [ ]:
# X_trainとy_trainの行数が、X_train, X_testのカラム数がそれぞれ同数なのでOK
print(X.shape, y.shape, X_test2.shape)

# 簡単に学習・予測・サブミットまでしてみる

## 学習

特徴量を作成できたので次にモデルの学習を行っていきます。この時大事になるのが交差検証 (Cross Validation) という考え方です。

* Cross Validation とは

Cross Validation とは学習用のデータセットを複数に分割してそれぞれの分割で学習・検証のデータセットを作り、モデルの性能を見積もる枠組みのことです。

* なんで Cross Validation するの?

なぜわざわざ分割するの? (そのまま全部学習で使っちゃえばいいじゃない?) と思われるのが普通だと思います。なぜ分割するかというと学習データの中で今の枠組みの性能(枠組みと言っているのは特徴量・モデルの構成もろもろ全部が含まれるためです)を評価したいからです。手元で評価ができないとLBに出してみて一喜一憂するしかなくなり、結果publicLBにオーバーフィットしてしまうので良くないです。

>仕事的な観点で言ってもLBに出すというのはデプロイ(本番へ反映すること)だから、本番に出さないとモデルの良し悪しがわからないのはよろしくないのと一緒

一番ナイーブな戦略は Random と呼ばれるものです。これは何も考えずにとにかくランダムに学習データを分割します。

その他にターゲットの分布が同じになるように分割する Stratified と呼ばれる方法もあります。

あとは「各分割で特定のグループが重ならないようにする」Group もよく使われます。

その他にも時系列で区切る TimeSeriesSplit という方法もあります。

### どの分割方法がいいの?
どの分割方法が一番良いのかは一概に言えません。

まず良い分割とはなにかを考えてみると、良い分割とは今のモデルがテストデータでどのぐらいの性能を出すかを、検証データで確認できる分割だと考えられます。

つまり、分割された学習/検証用データが、全体の学習/テストデータとの対応関係と一致していることといえます。
したがって本来、分割を決める際には学習データとテストデータの関係性をしらべ、それと同じ分割を採用する必要があります。

In [ ]:
# 今回は王道の Stratified K Fold (層化抽出)を利用します
## stratified K Foldはテストデータに含まれる各クラスの割合は、学習データに含まれる各クラスの割合とほぼ同じであろうという仮説に基づき、バリデーションの評価を安定させようとする手法です。
## 多クラス分類のような極端に頻度の少ないクラスがある場合は、層化抽出を行うのが重要です。ただ今回のような二値分類で偏りが大きくない場合はあまり効用を感じないかもしれないです。
from sklearn.model_selection import StratifiedKFold

fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=510)
cv = fold.split(X, y)
# split の返り値は generator だから、list 化して何度も iterate できるようにしておく
cv = list(cv)

## モデル構築

今回はlightgbmを使用します。
GBDT(Gradient Boosting Decision Tree: 勾配ブースティング木)と呼ばれる決定技をベースとしたアルゴリズムの一種でテーブルデータで性能が高いことが知られています。

lightgbmの特徴として
* 数値の大きさ自体に意味がなく、大小関係のみが影響する
* 欠損値が存在している場合にも自然に取り扱えるため特に処理が必要ない
* 決定技の分岐の繰り返しによって、変数間の相互作用を反映する
* 特徴重要度(`feature importance`)をさっと確認できる
* CPU 環境でも高速に学習・推論が行える

ほかにも理由はありますが u++ さんの [「初手LightGBM」をする7つの理由](https://upura.hatenablog.com/entry/2019/10/29/184617) などが参考になります

### LightGBM による CrossValidation を用いた学習

こちらも参考に\
[LightGBMで交差検証を実装してみるよ](https://potesara-tips.com/lightgbm-k-fold-cross-validation/)

In [ ]:
from sklearn.metrics import roc_auc_score
import lightgbm as lgbm

def fit_lgbm(X, y, 
                cv, 
                params: dict=None, 
                verbose: int=50,
                seed=random_seed):

    """lightGBM を CrossValidation の枠組みで学習を行うための関数を定義します"""

    # パラメータがない時は、空の dict で置き換える
    if params is None:
        params = {}
    
    models = []
    n_records = len(X)
    # training data の target と同じだけのゼロ配列を用意
    oof_pred = np.zeros((n_records,), dtype=np.float32)

    for i, (idx_train, idx_valid) in enumerate(cv):
        # この部分が交差検証のところ。データセットを `cv instance` によって分割します
        # training data を train/valid に分割
        x_train, y_train = X[idx_train], y[idx_train]
        x_valid, y_valid = X[idx_valid], y[idx_valid]

        clf = lgbm.LGBMClassifier(**params)

        with Timer(prefix='fit fold={}'.format(i)):
            clf.fit(x_train,
                    y_train,
                    eval_set = [(x_valid, y_valid)],
                    early_stopping_rounds=100,
                    verbose=verbose)
        
        pred_i = clf.predict_proba(x_valid)[:, 1]
        oof_pred[idx_valid] = pred_i
        models.append(clf)

        # 今回の指標の `roc_auc_score` で計算する
        score = roc_auc_score(y_valid, pred_i)
        print(f'{score:.4f}')

    score = roc_auc_score(y, oof_pred)
    print(f'{score:.4f}')
    
    return oof_pred, models

### parameter について
LightGBM などの GBDT のパラメータは、そこまでセンシティブではないです。しかし、内部的にどういう意味を持つのかを知っておくと、問題ごとにどういうパラメータが良いかの感覚がわかったり、チューニングする際にも有効なパラメータに絞ってチューニングできるので、重要な変数に関してはその意味についてざっと目を通しておくことと良いと言われています。

参考文献。

[LightGBM 徹底入門 – LightGBMの使い方や仕組み、XGBoostとの違いについて](https://www.codexa.net/lightgbm-beginner/)\
[Parameters Tuning](https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html): lightGBM 公式のパラメータチューニングガイド。英語。\
[勾配ブースティングで大事なパラメータの気持ち](https://bit.ly/3L2xmcN): gotoさんが書いた記事。日本語。

In [ ]:
lgbm_params = {
    # 目的関数、これの意味で最小となるようなパラメータを探します
    'objective': 'binary',

    # 学習率, 小さいほど滑らかな決定境界が作られて性能向上につながる場合が多いです
    # 一方でそれだけ木をつくるため、学習に時間がかります
    'learning_rate': .1,

    # L2 Reguralization
    'reg_lambda': .1,

    # L1
    'reg_alpha': 0,

    # 木の深さ、深い木を許容するほどより複雑な交互作用を考慮することになります
    'max_depth': 5,

    # 木の最大数, early_stopping という枠組みで木の数は制御されるようにしているので、とても大きい値を指定しておきます
    'n_estimators': 10000,

    # 木を作る際に考慮する特徴量の割合. 1以下を指定すると特徴をランダムに欠落させます 
    # 小さくすることで満遍なく特徴を使うという効果があるそうです
    'colsample_samples': 10,

    # 最小分割でのデータ数. 小さいとより細かい粒度の分割方法を許容します
    'min_child_samples': 10,

    # bagging の頻度と割合
    'subsample_freq': 3,
    'subsample': .9,

    # 特徴重要度計算のロジック
    'importance_type': 'gain',
    'random_state': 71,

}

## モデル評価

モデルを実行し、結果を見てみます。\
Cross Validationしたスコアの平均は最後に表示されており、訓練データで `0.7727` でした（乱数指定しているので毎回同じスコアが出ると思います）。\
lightgbmを使うと、それなりのスコアを出せることがわかります。\
サブミットしていないのでかなり安直ですが、メダル圏内まであと `0.02` ほど必要なので、まだまだ改善の余地があります。

In [ ]:
oof, models =fit_lgbm(X=X, y=y, cv=cv, params=lgbm_params)

### 特徴重要度（feature importance） の確認

In [ ]:
def visualize_importance(models, X_train):
    """lightGBM の model 配列の feature_importance を plot する関数です
    CVごとのブレを boxen plot として表現します

    args:
        models:
            List of lightGBM models
        X_train:
            学習時に使った DataFrame
    """

    feature_importance_df = pd.DataFrame()
    for i, model in enumerate(models):
        _df = pd.DataFrame()
        _df['feature_importance'] = model.feature_importances_
        _df['column'] = X_train.columns
        _df['fold'] = i + 1
        feature_importance_df = pd.concat(
            [feature_importance_df, _df],
            axis=0,
            ignore_index=True
        )
    
    order = feature_importance_df.groupby('column')\
        .sum()[['feature_importance']]\
        .sort_values('feature_importance', ascending=False).index[:50]

    
    fig, ax = plt.subplots(figsize=(8, max(6, len(order) * .25)))
    sns.boxenplot(data=feature_importance_df,
                  x='feature_importance',
                  y='column',
                  order=order,
                  ax=ax,
                  palette='viridis',
                  orient='h'
                  )

    ax.tick_params(axis='x', rotation=90)
    ax.set_title('Importance')
    ax.grid()
    fig.tight_layout()
    return fig, ax

以下のコードで重要度が可視化できます。\
feature_importance は値が大きいほど有効な分割であることを意味します。\
しかし、想定とは異なる部分が重要となっています（EXT_SOURCE_#ってなんや！？）\
てきとーにカラムを選択すると特徴重要度を見たときに有用な示唆を得にくくなる例ですね汗\
カラムを取捨選択したり、掛け合わせて作成したり、自分なりの仮説を持って特徴量生成をした後に再度可視化をするとまた面白い示唆を得られるかもしれないです。

In [ ]:
%matplotlib inline
fig, ax = visualize_importance(models, X_train)

In [ ]:
models

### 訓練データとテストデータでの予測結果の傾向差を見る
テストデータではどのような予測結果が出されるのか可視化してみます。\
今回のデータは訓練データとテストデータでユーザー特性が大きく異なることはあまりないと考えられます。\
なので訓練データとテストデータをもとにした予測結果もある程度は近くなると予測されます。

In [ ]:
# K 個のモデルの予測確率（predict_proba） を作成します。 shape = (k, N_test, n_classes) になるはずです。
pred_prob = np.array([model.predict_proba(X_test2) for model in models])
print(f"1. shape: {pred_prob.shape}")

# k 個のモデルの平均を計算
pred_prob = np.mean(pred_prob, axis=0) # axis=0 なので shape の `k` が潰れます
print(f"2. shape: {pred_prob.shape}")


# 欲しいのは y=1 の確率なので全要素の 1 次元目を取ってきます
pred_prob = pred_prob[:, 1]
print(f'3. shape: {pred_prob.shape}')

# ついでにsample_submissionのshapeとも比較しましょう
print('4. shape:', samp_sub.shape)

In [ ]:
pred_prob

どういったラベルが予測されているか、などの傾向を知っておきましょう。\
また学習時とテスト時で出力の乖離が無いか、を見ることも大事です。乖離が大きい場合には、入力する値自体が大きく異なっているなどで性能悪化が起こっている可能性があるのでサブミットする前に注意です。

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(10, 6))

sns.distplot(pred_prob, ax=ax, label="Test")
sns.distplot(oof, ax=ax, label="Out Of Fold (Train)")

ax.legend()
ax.grid()

#### NOTE: テストでの乖離が大きい とは
テストの予測値の乖離が大きい場合の原因はいくつか考えられますが「テスト時に使えない情報を特徴量としてつかっていないか?」を最も警戒した方が良いでう。

テスト時に使えない特徴 A を利用してモデルを作っていると、学習時に A をみるようなモデルが出来る可能性があり、テスト時にそれを参照できないことで予測が上手く行かない場合があります。

もっとも極端なのは A が予測ラベルそのものである場合です。学習時は予測ラベルを参照できるため、それこそ精度100%で予測できるようなモデルができますが、テスト時には当然予測ラベルはわからないため、精度は大きく悪化します。このように予測ラベルの情報が学習時の特徴量に染み出してしまった結果学習が上手く行かないことをリークとよびます（今回は予測対象が `TARGET` と分かりやすいので大丈夫ですね）。

テストでの乖離が起こるのはリークの場合だけではないですが、あまりに大きく異なる場合にはリークを含め、特徴量の選定に問題がないかを検討しましょう。

### サブミットファイルの作成

In [ ]:
samp_sub2 = samp_sub.copy()
samp_sub2.TARGET = pred_prob
samp_sub2.head()

In [ ]:
samp_sub2.to_csv(os.path.join(OUTPUT_DIR, 'submission.csv'), index=False)

In [ ]:
for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# 機械学習モデルを改善してみる

ここからスコアを上げるためにやることが大きく分けて3つあります。

雑な共有メモですが、、

* 特にGBDTでは差や比率を直接表現することが苦手なので明示的に作成したほうが精度が向上することがあります．
* 一方で大量に特徴量を作成した後に，LightGBMなどで学習させて，そのfeature importanceの上位の変数だけ用いるといったことをされる方もいます
* そもそも決定木を使用する際は，特徴量に対する対数化や，0から1の範囲に正規化するような大小関係が保存される変換の影響はほとんどありません．数値の大小関係で学習するモデルであるため，基本的にスケーリングを行う必要がありません．
* 一方で，線形回帰などはスケールの大きい変数ほど回帰係数が小さくなり，正則化がかかりにくいといった問題が生じてしまうので，NN含めスケーリングを行ったほうがいいことが多いです．(後述しますが二値変数や疎ベクトルに対してはスケーリングをしないほうが良い場合もあります．)

https://zenn.dev/colum2131/articles/fffac4654e7c7c

https://www.nogawanogawa.com/entry/mlflow_lgbm

https://kakeami.github.io/viz-madb/index.html

https://github.com/FavioVazquez/ds-cheatsheets/blob/master/Python/Others/mementopython3-english.pdf

## 前処理

## EDA（探索的データ分析）& 特徴量生成

## モデル選択 & パラメーターチューニング